In [ ]:
# @title Vid2densepose
from IPython.display import clear_output, display, HTML

%cd /content
!wget https://huggingface.co/camenduru/densepose/raw/main/Base-DensePose-RCNN-FPN.yaml -O /content/Base-DensePose-RCNN-FPN.yaml
!wget https://huggingface.co/camenduru/densepose/raw/main/densepose_rcnn_R_50_FPN_s1x.yaml -O /content/densepose_rcnn_R_50_FPN_s1x.yaml
!pip install -q gradio
!pip install -q https://huggingface.co/camenduru/densepose/resolve/main/detectron2-0.6-cp310-cp310-linux_x86_64.whl?download=true
!pip install -q https://huggingface.co/camenduru/densepose/resolve/main/detectron2_densepose-0.6-py3-none-any.whl?download=true

# https://github.com/Flode-Labs/vid2densepose/blob/main/app.py modified
import gradio as gr
from detectron2.config import get_cfg
import torch
import cv2
import numpy as np
from detectron2.engine import DefaultPredictor
from densepose import add_densepose_config
from densepose.vis.extractor import DensePoseResultExtractor
from densepose.vis.densepose_results import DensePoseResultsFineSegmentationVisualizer as Visualizer
import tempfile
import shutil

def process_video(input_video_path):
    output_video_path = tempfile.NamedTemporaryFile(suffix=".mp4", delete=False).name
    cfg = get_cfg()
    add_densepose_config(cfg)
    cfg.merge_from_file("/content/densepose_rcnn_R_50_FPN_s1x.yaml")
    predictor = DefaultPredictor(cfg)
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        with torch.no_grad():
            outputs = predictor(frame)['instances']
        results = DensePoseResultExtractor()(outputs)
        cmap = cv2.COLORMAP_VIRIDIS
        arr = cv2.applyColorMap(np.zeros((height, width), dtype=np.uint8), cmap)
        out_frame = Visualizer(alpha=1, cmap=cmap).visualize(arr, results)
        out.write(out_frame)
    cap.release()
    out.release()
    return output_video_path
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="Input Video"),
    outputs=gr.Video(label="Output DensePose Video"),
    title="Video 2 DensePose"
)
clear_output()
iface.launch(share=True, inline=False, debug=True)

In [ ]:
iface.launch(share=True, inline=False, debug=True)

In [ ]:
# @title MagicAnimat
from IPython.display import clear_output, display, HTML

%cd /content
!git clone https://github.com/Van-wise/magicanimate-colab MagicAnimate-hf
!pip install typing-extensions==4.5.0
!pip install -q diffusers==0.21.4 transformers==4.32.0 accelerate==0.22.0 omegaconf==2.3.0 einops==0.6.1 av gradio
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

!apt -y install -qq aria2
SDVAE_Url = "https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main"
SDVAE_Dir = "/content/MagicAnimate-hf/sd-vae-ft-mse"
MA_Url = "https://huggingface.co/zcxu-eric/MagicAnimate/resolve/main"
MA_Dir = "/content/MagicAnimate-hf/MagicAnimate"
SD15_Url = "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main"
SD15_Dir = "/content/MagicAnimate-hf/stable-diffusion-v1-5"

import subprocess
from concurrent import futures

def download_file(file_dLlink, save_dir, file_name):
    try:
        #subprocess.run(['aria2c', '--console-log-level=error', '-q', '-c', '-x', '16', '-s', '16', '-k', '1M', file_dLlink, '-d', save_dir, '-o', file_name], check=True)
        !aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {file_dLlink} -d {save_dir} -o {file_name}
        print(f"File {save_dir}/{file_name} downloaded successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading file {file_name}: {e}")

# 定义要下载的文件列表
file_list = [
    (SD15_Url + "/v1-5-pruned-emaonly.safetensors", SD15_Dir, "v1-5-pruned-emaonly.safetensors"),
    (SDVAE_Url + "/diffusion_pytorch_model.bin", SDVAE_Dir, "diffusion_pytorch_model.bin"),
    (SDVAE_Url + "/diffusion_pytorch_model.safetensors", SDVAE_Dir, "diffusion_pytorch_model.safetensors"),
    (SDVAE_Url + "/config.json", SDVAE_Dir, "config.json"),
    (MA_Url + "/appearance_encoder/diffusion_pytorch_model.safetensors", MA_Dir + "/appearance_encoder", "diffusion_pytorch_model.safetensors"),
    (MA_Url + "/appearance_encoder/config.json", MA_Dir + "/appearance_encoder", "config.json"),
    (MA_Url + "/densepose_controlnet/diffusion_pytorch_model.safetensors", MA_Dir + "/densepose_controlnet", "diffusion_pytorch_model.safetensors"),
    (MA_Url + "/densepose_controlnet/config.json", MA_Dir + "/densepose_controlnet", "config.json"),
    (MA_Url + "/temporal_attention/temporal_attention.ckpt", MA_Dir + "/temporal_attention", "temporal_attention.ckpt"),
    (SD15_Url + "/feature_extractor/preprocessor_config.json", SD15_Dir + "/feature_extractor", "preprocessor_config.json"),
    (SD15_Url + "/safety_checker/pytorch_model.bin", SD15_Dir + "/safety_checker", "pytorch_model.bin"),
    (SD15_Url + "/safety_checker/config.json", SD15_Dir + "/safety_checker", "config.json"),
    (SD15_Url + "/scheduler/scheduler_config.json", SD15_Dir + "/scheduler", "scheduler_config.json"),
    (SD15_Url + "/text_encoder/pytorch_model.bin", SD15_Dir + "/text_encoder", "pytorch_model.bin"),
    (SD15_Url + "/text_encoder/config.json", SD15_Dir + "/text_encoder", "config.json"),
    (SD15_Url + "/tokenizer/merges.txt", SD15_Dir + "/tokenizer", "merges.txt"),
    (SD15_Url + "/tokenizer/special_tokens_map.json", SD15_Dir + "/tokenizer", "special_tokens_map.json"),
    (SD15_Url + "/tokenizer/tokenizer_config.json", SD15_Dir + "/tokenizer", "tokenizer_config.json"),
    (SD15_Url + "/tokenizer/vocab.json", SD15_Dir + "/tokenizer", "vocab.json"),
    (SD15_Url + "/unet/diffusion_pytorch_model.bin", SD15_Dir + "/unet", "diffusion_pytorch_model.bin"),
    (SD15_Url + "/unet/config.json", SD15_Dir + "/unet", "config.json"),
    (SD15_Url + "/vae/diffusion_pytorch_model.bin", SD15_Dir + "/vae", "diffusion_pytorch_model.bin"),
    (SD15_Url + "/vae/config.json", SD15_Dir + "/vae", "config.json"),
    (SD15_Url + "/model_index.json", SD15_Dir, "model_index.json"),
    (SD15_Url + "/v1-inference.yaml", SD15_Dir, "v1-inference.yaml")
]

# 创建线程池并下载文件
with futures.ThreadPoolExecutor() as executor:
    download_tasks = [executor.submit(download_file, file_dLlink, save_dir, file_name) for file_dLlink, save_dir, file_name in file_list]

    # 等待所有下载任务完成
    for future in futures.as_completed(download_tasks):
        try:
            future.result()
        except Exception as e:
            print(f"Error downloading file: {e}")
clear_output()

In [ ]:
#@markdown #####<font color="#2c9678">SD1.5模型下载链接:
SD15_link = "https://civitai.com/api/download/models/11745" # @param {type:"string"}
!rm -rf {SD15_Dir}/v1-5-pruned-emaonly.safetensors
!aria2c --console-log-level=error -q -c -x 16 -s 16 -k 1M {SD15_link} -d {SD15_Dir}/v1-5-pruned.safetensors -o v1-5-pruned.safetensors


In [ ]:
%cd /content/MagicAnimate-hf
!python app.py